In [1]:
!pip install transformers datasets evaluate sacrebleu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 16.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [2]:
from huggingface_hub import login
from google.colab import userdata
from datasets import load_dataset

login(userdata.get('HF_READ'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
# https://huggingface.co/datasets/Helsinki-NLP/opus-100
test_es = load_dataset("Helsinki-NLP/opus-100", "en-es")["test"]
test_ja = load_dataset("Helsinki-NLP/opus-100", "en-ja")["test"]
test_zh = load_dataset("Helsinki-NLP/opus-100", "en-zh")["test"]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [4]:
import datasets
lang_dict = {"es": "Spanish", "zh": "Chinese", "ja": "Japanese"}

lang = 'es'
def preprocess(example):
  source = lang_dict[lang]
  t = example['translation']
  prefix = f"translate {source} to English: "

  input = prefix + t[lang]
  target = t['en']

  return {"text": input, 'label': target}

proc_es = test_es.map(preprocess, remove_columns='translation')
lang = 'ja'
proc_ja = test_ja.map(preprocess, remove_columns='translation')

lang = 'zh'
proc_zh = test_zh.map(preprocess, remove_columns='translation')

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [5]:
from transformers import pipeline
check = "lseely916/CMSC_473_mt5"
baseline = "google-t5/t5-small"
max_len = 512
es_mt5 = pipeline(f"translation_es_to_en", model=check, max_length=max_len, device=0)
ja_mt5 = pipeline(f"translation_ja_to_en", model=check, max_length=max_len, device=0)
zh_mt5 = pipeline(f"translation_zh_to_en", model=check, max_length=max_len, device=0)

es_t5 = pipeline(f"translation_es_to_en", model=baseline, max_length=max_len, device=0)
ja_t5 = pipeline(f"translation_ja_to_en", model=baseline, max_length=max_len, device=0)
zh_t5 = pipeline(f"translation_zh_to_en", model=baseline, max_length=max_len, device=0)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/861 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [6]:
import evaluate
from evaluate import evaluator

metric = evaluate.load("sacrebleu")
task_eval = evaluator("translation")
def eval_pipe(p, d, lang):
  eval_results = task_eval.compute(
    model_or_pipeline=p,
    data=d
  )
  print(eval_results)

In [7]:
eval_pipe(es_mt5, proc_es, 'es')
eval_pipe(ja_mt5, proc_ja, 'ja')
eval_pipe(zh_mt5, proc_zh, 'zh')


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'bleu': 0.14101975670752345, 'precisions': [0.4176132966638766, 0.17887574717029123, 0.09719911729757257, 0.05446678544158819], 'brevity_penalty': 1.0, 'length_ratio': 1.1132113144758735, 'translation_length': 33452, 'reference_length': 30050, 'total_time_in_seconds': 727.32699834, 'samples_per_second': 2.7497948028392445, 'latency_in_seconds': 0.36366349917}


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'bleu': 0.055416396638815694, 'precisions': [0.2606808756871444, 0.07247304941829437, 0.03171474646120767, 0.01574005269202189], 'brevity_penalty': 1.0, 'length_ratio': 1.034572212521826, 'translation_length': 20738, 'reference_length': 20045, 'total_time_in_seconds': 533.0320887759999, 'samples_per_second': 3.752119322858394, 'latency_in_seconds': 0.266516044388}


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'bleu': 0.2088784473236807, 'precisions': [0.4828033489249636, 0.24542776482140666, 0.15332935204538667, 0.10477441767247138], 'brevity_penalty': 1.0, 'length_ratio': 1.0212962405411372, 'translation_length': 50882, 'reference_length': 49821, 'total_time_in_seconds': 1105.719381663, 'samples_per_second': 1.8087771935335017, 'latency_in_seconds': 0.5528596908315}


In [17]:
eval_pipe(es_t5, proc_es, 'es')
eval_pipe(ja_t5, proc_ja, 'ja')
eval_pipe(zh_t5, proc_zh, 'zh')

{'bleu': 0.0274279101342105, 'precisions': [0.14143530644316396, 0.03662735646035982, 0.020361990950226245, 0.01247081035425051], 'brevity_penalty': 0.8098845915995693, 'length_ratio': 0.8258569051580699, 'translation_length': 24817, 'reference_length': 30050, 'total_time_in_seconds': 1020.3588769019998, 'samples_per_second': 1.9600946738194445, 'latency_in_seconds': 0.5101794384509999}
{'bleu': 2.4396667990950237e-07, 'precisions': [0.5014577259475219, 0.18633540372670807, 0.1598173515981735, 0.10429447852760736], 'brevity_penalty': 1.228080337407122e-06, 'length_ratio': 0.06844599650785732, 'translation_length': 1372, 'reference_length': 20045, 'total_time_in_seconds': 143.31929699900047, 'samples_per_second': 13.954854941926964, 'latency_in_seconds': 0.07165964849950024}
{'bleu': 0.0006720578586099761, 'precisions': [0.5872406933788008, 0.2648825778263244, 0.23054029304029305, 0.20657015590200445], 'brevity_penalty': 0.0022907920908801505, 'length_ratio': 0.14126573131811887, 'trans